# ChatBot to Provide Flight Details from Conversational Text

## Steps

* Import necessary libraries
* Load raw data in a json format (intents.json)
* Preprocess raw data to get training data
* Train data using tflearn DNN model, and save model
* Create a function to take input texts and return output texts based on model predictions of trained responses
* Start talking with the Bot

### Import Necessary Libraries

In [1]:
import warnings
warnings.filterwarnings("ignore")

import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import numpy as np
import pandas as pd
import tflearn
import tensorflow
from tensorflow import keras
import random
import json
import pickle
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

nltk.download('punkt')
nltk.download('wordnet')


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to /home/learner/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/learner/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Load Raw Data in .json Format (intents.json)

In [102]:
with open("intents.json") as file:
    data = json.load(file)

### Preprocess raw data to get training data

In [103]:
words = []
labels = []
docs_x = []
docs_y = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent['tag'])
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
words = [stemmer.stem(w.lower()) for w in words if w not in ['?', '!']]
words = sorted(list(set(words)))
labels = sorted(labels)

training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds_doc = [stemmer.stem(wd) for wd in doc]

    for _ in words:
        if _ in wrds_doc:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)


training = np.array(training)
output = np.array(output)

### Train data using tflearn DNN model, and save model

In [106]:
tensorflow.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

model.fit(training, output, n_epoch=200, batch_size=8, show_metric=True)
model.save("model.ftlearn")

Training Step: 1399  | total loss: 0.09176 | time: 0.038s
| Adam | epoch: 200 | loss: 0.09176 - acc: 1.0000 -- iter: 48/52
Training Step: 1400  | total loss: 0.08769 | time: 0.043s
| Adam | epoch: 200 | loss: 0.08769 - acc: 1.0000 -- iter: 52/52
--
INFO:tensorflow:/home/learner/Documents/unit6_project/model.ftlearn is not in all_model_checkpoint_paths. Manually adding it.


### Create a function to take input texts and return output texts based on model predictions of trained responses

In [110]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]
    
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]
    
    for se in s_words:
        for i, wd in enumerate(words):
            if wd == se:
                bag[i] = 1
    return np.array(bag)

def chat():
    print("Start talking with the bot (type quit to stop)")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break
        results = model.predict([bag_of_words(inp, words)])
        results_index = np.argmax(results)
        tag = labels[results_index]
        
        for tg in data['intents']:
            if tg['tag'] == tag:
                responses = tg['responses']
        
        print(f'Bot: {random.choice(responses)}')

In [ ]:
chat()

Start talking with the bot (type quit to stop)
You: hi
Bot: Hello!
You: are you ther
Bot: Hi there


In [ ]:
for intent in data['intents']:
    for pattern in intent['patterns']:

In [98]:
for tg in data['intents']:
    print(tg['responses'])

['Hello!', 'Good to see you again', 'Hi there', 'how can i help you']
['San Francisco', 'Chicago', 'Los Angeles', 'Chiang Mai, Thailand', 'Portland', 'Sapporo, Japan', 'Phoenix', 'Anchorage, Alaska', 'New York', 'Santiago, Chile', 'Mendoza, Argentina', 'Philadelphia', 'Jakarta, Indonesia', 'Seattle', 'Nagoya, Japan', 'Dallas', 'Sapporo, Japan', 'Dallas', 'Chicago', 'Quito, Ecuador', 'Miami', 'Cuzco, Peru', 'Boston', 'Santa Barbara, California', 'Monterey, California', 'Palm Springs', 'Baltimore', 'Phoenix', 'San Jose, California', 'Maui, Hawaii', 'San Antonio', 'Miami', 'Vancouver, Canada', 'Johannesburg, South Africa', 'San Diego', 'Washington D.C.', 'San Francisco', 'Mumbai, India', ' New Delhi, India', 'Warsaw, Poland', 'San Juan, Puerto Rico', 'New Orleans', 'Hanoi, Vietnam', 'Singapore', 'Las Vegas', 'Orlando']
['$564 for roundtrip', '$96 - Basic economy', '$254 - Regular economy', 'from $200 basic economy to $273 regular economy', '$203', '$430', '$602', 'fares range from $186 to

KeyError: 'responses'

In [ ]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]
    
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

In [92]:
ife = 'hello and how are you   '

In [42]:
[0 for _ in range(len(ife))]

[0, 0, 0, 0, 0]

In [52]:
nltk.word_tokenize(ife)

['hello', ',', 'and', 'how', 'are', 'you']

In [95]:
ife.strip().split(" ")

['hello', 'and', 'how', 'are', 'you']

In [56]:
#[lemmatizer.lemmatize(w.lower()) for w in ife]

In [62]:
print(lemmatizer.lemmatize(ife))

hello, and how are you


In [64]:
print(stemmer.stem(ife))

hello, and how are you


In [83]:
fams = {"name": ["Ugo", "Ud", "Ifea", "prec", "joy"]} #,
       #"age": [42,40,38,36,34]}
person = input("get info for: ")

print(f' {person} is my {fams[person]}')
               

get info for: Ugo


KeyError: 'Ugo'

In [75]:
fams['ag']

KeyError: 'ag'